# <a id='toc1_'></a>[Building LLM Agents - Red Cross Helpful Information as Aid](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Building LLM Agents - Red Cross Helpful Information as Aid](#toc1_)    
  - [Introduction to LLM Agents](#toc1_1_)    
    - [What Are LLM Agents?](#toc1_1_1_)    
    - [How Do LLM Agents Work?](#toc1_1_2_)    
  - [Building a Basic LLM Agent with OpenAI](#toc1_2_)    
    - [Prompt Engineering](#toc1_2_1_)    
    - [Structured Output](#toc1_2_2_)    
  - [Enhancing LLM Agents with LangChain](#toc1_3_)    
    - [Simple Agent](#toc1_3_1_)    
    - [Web Search Agent](#toc1_3_2_)    
  - [Multi-Agent Systems with LangGraph](#toc1_4_)    
    - [What's a DAG?](#toc1_4_1_)    
      - [Graph](#toc1_4_1_1_)    
      - [Acyclic](#toc1_4_1_2_)    
      - [Directed](#toc1_4_1_3_)    
    - [Where else are DAGs used?](#toc1_4_2_)    
    - [Why Are DAGs Important for LLM Agents?](#toc1_4_3_)    
    - [Building our own DAG - a simple multi-agent system](#toc1_4_4_)    
  - [Conclusion](#toc1_5_)    
- [Extra](#toc2_)    
  - [Tool-Using Agent](#toc2_1_)    
  - [RAG Agent](#toc2_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Introduction to LLM Agents](#toc0_)

### <a id='toc1_1_1_'></a>[What Are LLM Agents?](#toc0_)
> Agents are systems that use LLMs as **reasoning engines** to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling. *(Source: [LangChain](https://python.langchain.com/docs/tutorials/agents/))*

### <a id='toc1_1_2_'></a>[How Do LLM Agents Work?](#toc0_)
Just like any other LLM. The only difference is that based on the constraints we put on LLM agents (e.g. prompt, structured output), we are able to get them to fulfill a specific role without further model finetuning. An LLM agent typically sits within a pipeline of LLM agents, like we see in [this example](https://langchain-mrkl.streamlit.app/?ref=streamlit-io-gallery-llms). 

## <a id='toc1_2_'></a>[Building a Basic LLM Agent with OpenAI](#toc0_)
Ensure you have the OpenAI Python library installed and use a `.env` file to set up your API key (`OPENAI_API_KEY`).

In [ ]:
# !pip install openai
# !pip install dotenv

In [44]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Set up your API key and define a simple LLM agent function:


In [45]:
message_history_1 = [{"role": "system", "content": "You are a helpful assistant"}]

In [46]:
import openai
import os 
from pprint import pprint

def llm_agent(input_, message_history, model="gpt-3.5-turbo"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.chat.completions.create(
      model=model,
      messages=message_history
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [47]:
pprint(llm_agent("I am hungry", message_history_1))

[{'content': 'You are a helpful assistant', 'role': 'system'},
 {'content': 'I am hungry', 'role': 'user'},
 {'content': 'I can help you find a recipe to make a meal at home, suggest '
             'nearby restaurants for takeout or delivery, or recommend snack '
             "ideas you may have on hand. Just let me know what you're in the "
             'mood for!',
  'role': 'assistant'}]


While this is not inherently a bad reply, it doesn't provide any specific support right away. So during this lesson, we'll try to make this better, bit by bit.

### <a id='toc1_2_1_'></a>[Prompt Engineering](#toc0_)

We'll aim to improve our assistant and reproduce a small part of [HIA (Helpful Information as Aid)](https://github.com/sabinagio/HIA) - a multi-agent solution for the Red Cross 510 data team:

In [48]:
domain_list = [
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]

In [49]:
red_cross_assistant_prompt = """You are an expert query analyzer for a Red Cross virtual assistant.
    Analyze the query to understand the user's needs, emotional state, and language.
    Pay special attention to any signs of emergency or urgent needs.

    If you detect any of these, mark as EMERGENCY:
    - Immediate danger
    - Medical emergencies 
    - Severe distress
    - Threats to basic safety

    If the query is unclear, you should return relevant domains from this list as clarification options:
    {}

    Important: You will analyze:
    1. Query clarity and type (clear/needs_clarification/emergency)
    2. Domains of need, from this list of options {} or "Other"
    3. Emotional state from language and content
    4. Language of query
    """.format(domain_list, domain_list)

In [50]:
message_history_2 = [{"role": "system", "content": red_cross_assistant_prompt}]

In [51]:
pprint(llm_agent("I am hungry", message_history_2))

[{'content': 'You are an expert query analyzer for a Red Cross virtual '
             'assistant.\n'
             "    Analyze the query to understand the user's needs, emotional "
             'state, and language.\n'
             '    Pay special attention to any signs of emergency or urgent '
             'needs.\n'
             '\n'
             '    If you detect any of these, mark as EMERGENCY:\n'
             '    - Immediate danger\n'
             '    - Medical emergencies \n'
             '    - Severe distress\n'
             '    - Threats to basic safety\n'
             '\n'
             '    If the query is unclear, you should return relevant domains '
             'from this list as clarification options:\n'
             "    ['Shelter', 'Health & Wellbeing', 'Safety & Protection']\n"
             '\n'
             '    Important: You will analyze:\n'
             '    1. Query clarity and type '
             '(clear/needs_clarification/emergency)\n'
             "    2.

Now the issue with this is that it doesn't provide any useful information, it just asks clarifying questions. But that's ok, we can create an LLM with...

### <a id='toc1_2_2_'></a>[Structured Output](#toc0_)

In [ ]:
# !pip install pydantic

In [52]:
from typing import List, Literal
from typing_extensions import Union
from pydantic import BaseModel # pydantic is a data validation library widely used in the industry

Domains = Literal[
    "Shelter",
    "Health & Wellbeing",
    "Safety & Protection",
]
DomainWithOther = Union[Domains, Literal["Other"]]

class QueryAnalysis(BaseModel):
    """Analysis output from Query Understanding Agent"""
    original_query: str
    query_type: Literal["clear", "emergency"]
    domains: List[DomainWithOther]
    emotional_state: str
    language: str 

# QueryAnalysis()

In [53]:
def llm_agent(input_, message_history, model="gpt-4o-2024-08-06"):
    llm = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    message_history.append({"role": "user", "content": f"{input_}"})

    # Generate a response from the chatbot model
    completion = llm.beta.chat.completions.parse(
      model=model,
      messages=message_history,
      response_format=QueryAnalysis
    )

    # We save the assistant response
    message_history.append({"role": "assistant", "content": completion.choices[0].message.content})
    return message_history

In [54]:
pprint(llm_agent("I am hungry", message_history_2))

[{'content': 'You are an expert query analyzer for a Red Cross virtual '
             'assistant.\n'
             "    Analyze the query to understand the user's needs, emotional "
             'state, and language.\n'
             '    Pay special attention to any signs of emergency or urgent '
             'needs.\n'
             '\n'
             '    If you detect any of these, mark as EMERGENCY:\n'
             '    - Immediate danger\n'
             '    - Medical emergencies \n'
             '    - Severe distress\n'
             '    - Threats to basic safety\n'
             '\n'
             '    If the query is unclear, you should return relevant domains '
             'from this list as clarification options:\n'
             "    ['Shelter', 'Health & Wellbeing', 'Safety & Protection']\n"
             '\n'
             '    Important: You will analyze:\n'
             '    1. Query clarity and type '
             '(clear/needs_clarification/emergency)\n'
             "    2.

In [60]:
message_history_2[-1]['content']

'{"original_query":"I am hungry","query_type":"clear","domains":["Health & Wellbeing"],"emotional_state":"Basic need for food","language":"English"}'

In [61]:
pprint(llm_agent("I have nowhere to sleep", message_history_2))

[{'content': 'You are an expert query analyzer for a Red Cross virtual '
             'assistant.\n'
             "    Analyze the query to understand the user's needs, emotional "
             'state, and language.\n'
             '    Pay special attention to any signs of emergency or urgent '
             'needs.\n'
             '\n'
             '    If you detect any of these, mark as EMERGENCY:\n'
             '    - Immediate danger\n'
             '    - Medical emergencies \n'
             '    - Severe distress\n'
             '    - Threats to basic safety\n'
             '\n'
             '    If the query is unclear, you should return relevant domains '
             'from this list as clarification options:\n'
             "    ['Shelter', 'Health & Wellbeing', 'Safety & Protection']\n"
             '\n'
             '    Important: You will analyze:\n'
             '    1. Query clarity and type '
             '(clear/needs_clarification/emergency)\n'
             "    2.

Wonderful! We've already created a useful AI agent. However, if we wanted to switch to a different provider, e.g. Anthropic, or use an open-source LLM, e.g. Llama, DeepSeek, we would need to change our code completely! 

## <a id='toc1_3_'></a>[Enhancing LLM Agents with LangChain](#toc0_)

With LangChain, we can use a lot more models without changing our code each time we change an LLM. Additionally, LangChain is able to couple multiple agents together, allowing an LLM to answer more complex queries more accurately.

In [ ]:
# !pip install langchain
# !pip install langchain_community
# !pip install -U langchain-openai

### <a id='toc1_3_1_'></a>[Simple Agent](#toc0_)

In [62]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")
response = llm.invoke(message_history_1)
pprint(response)

AIMessage(content='What type of cuisine are you interested in?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 68, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-688b7563-fd73-45e9-9e8b-5eef5f418b6f-0', usage_metadata={'input_tokens': 68, 'output_tokens': 10, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [63]:
response = llm.invoke(message_history_2)
pprint(response)

AIMessage(content='1. Query clarity and type: Emergency\n2. Domains of need: Shelter\n3. Emotional state: Distressed\n4. Language of query: English\n\nThis query indicates an urgent need for shelter (Shelter domain) and the user seems to be in distress.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 369, 'total_tokens': 426, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-35b624f0-ad88-4064-83c7-c27deb0c4ff8-0', usage_metadata={'input_tokens': 369, 'output_tokens': 57, 'total_tokens': 426, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [64]:
response = llm.with_structured_output(QueryAnalysis).invoke(message_history_2)
pprint(response)

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


QueryAnalysis(original_query='I have nowhere to sleep', query_type='emergency', domains=['Shelter'], emotional_state='Distressed', language='English')


In [65]:
response.original_query

'I have nowhere to sleep'

In [66]:
response.language

'English'

In [67]:
response.query_type

'emergency'

Now I was able to do the exact same thing with significantly fewer lines of code! Now let's see what other agents we can play with.

### <a id='toc1_3_2_'></a>[Web Search Agent](#toc0_)

In [ ]:
# !pip install -U duckduckgo-search

In [69]:
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

def web_search(query: str) -> dict:
    wrapper = DuckDuckGoSearchAPIWrapper(region="nl-nl", max_results=2) # time='y' limit to past year (m, d, w)
    search_tool = DuckDuckGoSearchResults(api_wrapper=wrapper)
    return search_tool.run(query)

In [70]:
pprint(web_search("I am hungry"))

('snippet: The expression "Ik heb honger" is a common and neutral way to say '
 '"I\'m hungry" in both formal and casual contexts. There are no specific '
 'expressions that make the translation formal or casual in this context., '
 'title: How do you say "i\'m hungry " in Dutch? | HiNative, link: '
 'https://hinative.com/questions/26523530, snippet: Waking up hungry is more '
 "than just a cue that you're ready for breakfast. Three experts on the "
 "reasons you're waking up hungry and what it says about your health., title: "
 '8 Reasons You May Be Waking Up Hungry, According to Experts - TODAY, link: '
 'https://www.today.com/health/health/waking-up-hungry-rcna192814, snippet: '
 'Feeling hungry but lacking appetite can be frustrating. Learn the causes, '
 'from medical conditions to mental health, and discover tips to regain your '
 'desire to eat. For New Providers, title: Hungry, but No Appetite: Why it '
 'Happens & What to Do | Season, link: '
 'https://www.seasonhealth.com/blo

Awesome! We can search stuff on the internet now. Let's see how we can combine this with our previous agent to get a better search query.

In [71]:
system_prompt = """
    Create a simple search query (maximum 10 words) in English 
    that will help find relevant assistance information.
    Return ONLY the search query, no explanation or strategy.
    """

domain_sites = {
        "food": ["voedselbank.nl", "voedselbankennederland.nl"],
        "shelter": ["deregenboog.org", "opvang.nl"],
        "healthcare": ["ggd.nl", "zorgverzekeringslijn.nl"],
        "domestic_violence": ["veiligthuis.nl", "blijfgroep.nl"],
    }

In [72]:
def build_search_query(query: QueryAnalysis, domains=domain_sites, prompt=system_prompt):
    # Prepare relevant websites
    relevant_sites = ["rodekruis.nl"]
    if query.domains:
        for domain in domains:
            if domain.lower() in domain_sites:
                relevant_sites.extend(domain_sites[domain.lower()])

    # Generate search query using LLM
    llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")
    search_query_response = llm.invoke([
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"""
            Original query: {query.original_query}
            Language: {query.language}
            Domains: {query.domains or 'Not specified'}
        """}
    ])
    print("This is the LLM query search response:")
    pprint(search_query_response)
    search_query = search_query_response.content
    domain_priority = " OR ".join(f"site:{site}" for site in relevant_sites)
    return f"{search_query} {domain_priority}"

In [73]:
response = llm.with_structured_output(QueryAnalysis).invoke([
        {"role": "system", "content": red_cross_assistant_prompt},
        {"role": "user", "content": "I am hungry"},
    ]
)
pprint(response)

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


QueryAnalysis(original_query='I am hungry', query_type='clear', domains=['Health & Wellbeing'], emotional_state='Neutral', language='English')


In [74]:
web_response = web_search(build_search_query(response))
pprint(web_response)

This is the LLM query search response:
AIMessage(content='How to find food assistance nearby.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 74, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e06f96d4-596d-4f02-867c-cff3f3e5ef9b-0', usage_metadata={'input_tokens': 74, 'output_tokens': 8, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
('snippet: Je bent welkom. Heb je even geen geld om eten te kopen? Geen '
 'zorgen. Dat kan iedereen overkomen. Wanneer kom ik in aanmerking? Als je je '
 'aanmeldt, krijg je meteen boodschappen mee. Dat doen we minimaa

## <a id='toc1_4_'></a>[Multi-Agent Systems with LangGraph](#toc0_)
Although we can already build a multi-agent system with LangChain, we have one constraint: the sequence of actions can only be linear. LangGraph extends LangChain by enabling parallel/conditional interactions through DAGs (Directed Acyclic Graphs).

### <a id='toc1_4_1_'></a>[What's a DAG?](#toc0_)

A **Directed Acyclic Graph (DAG)** is a graph structure where nodes are connected by directed edges, and there are no cycles. This means that information flows in one direction without looping back.

#### <a id='toc1_4_1_1_'></a>[Graph](#toc0_)

A **graph** is a data structure consisting of nodes (or vertices) connected by edges. Graphs are widely used in computer science, mathematics, and AI to model relationships between entities.

![](../../../img/graph_edge_matrix.png)  
(Source: [HuggingFace](http://huggingface.co/blog/intro-graphml))

#### <a id='toc1_4_1_2_'></a>[Acyclic](#toc0_)

- **Cyclic vs. Acyclic Graphs**: A cyclic graph has cycles, whereas an acyclic graph does not.  

![](../../../img/a_cyclical_graphs.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

#### <a id='toc1_4_1_3_'></a>[Directed](#toc0_)

**Directed Graphs**: Edges have a specific direction (A → B).  

![](../../../img/directed-graph.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))

### <a id='toc1_4_2_'></a>[Where else are DAGs used?](#toc0_)

![](../../../img/airflow_dag.png)  
(Source: [Astronomer](https://www.astronomer.io/docs/learn/dags/))  

DAGs are widely used in streamlining data/ML operations that need to happen sequentially and at regular time intervals. Airflow is one of the most basic platforms where you can build this type of pipelines and monitor the success/failure of your pipelines. You can also schedule runs of pipelines, connect to cloud services, etc. Other schedulers used by data scientists are Prefect (recommended), Dask (for large-scale data processing), Kubeflow pipelines (can be overkill), DAGster. 

You'd do have to do some research before choosing any of these - Airflow is always a good starting point as it's open-source, but users complain that it's outdated compared to competitors.

### <a id='toc1_4_3_'></a>[Why Are DAGs Important for LLM Agents?](#toc0_)
- **Task Execution Flow**: DAGs define clear steps in multi-agent workflows.
- **Parallel Processing**: Independent tasks can run simultaneously.
- **Dependency Management**: Ensures tasks execute in the correct order.

LangGraph uses DAGs to structure multi-agent workflows, ensuring efficient and logical task progression.

### <a id='toc1_4_4_'></a>[Building our own DAG - a simple multi-agent system](#toc0_)

In [ ]:
# !pip install langgraph

In [75]:
from langgraph.graph import StateGraph, START, END

def query_understanding_node(state):
    llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")
    response = llm.with_structured_output(QueryAnalysis).invoke([
            {"role": "system", "content": red_cross_assistant_prompt},
            {"role": "user", "content": state["response"]},
        ]
    )
    return {"response": response}

def web_agent_node(state):
    response = web_search(build_search_query(query=state['response']))
    return {"response": response}

In [83]:
from typing_extensions import TypedDict

class ConversationBox(TypedDict):
    """State for the entire conversation graph"""
    response: str

workflow = StateGraph(ConversationBox)
workflow.add_node("query_understanding", query_understanding_node)
workflow.add_node("web_agent", web_agent_node)
workflow.add_edge(START, "query_understanding")
workflow.add_edge("query_understanding", "web_agent")
workflow.add_edge("web_agent", END)
graph = workflow.compile()

In [84]:
response = graph.invoke({"response": "Please help me!"})
pprint(response)

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


This is the LLM query search response:
AIMessage(content='Find emergency shelter assistance near me.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 83, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8aef92df-8419-41bc-8ae5-78fcc9f8666a-0', usage_metadata={'input_tokens': 83, 'output_tokens': 8, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
{'response': 'snippet: Een brandwond of vergiftiging, bewusteloos raken of '
             'iets kneuzen. Een ongeluk kan zomaar gebeuren. Met de gratis '
             'EHBO-app van het Rode Kruis weet je altijd e

In [78]:
response = graph.invoke({"response": "I am hungry"})
pprint(response)

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


This is the LLM query search response:
AIMessage(content='Find food assistance near me.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f249b124-67a4-4ca4-b220-adfdcc5c1a28-0', usage_metadata={'input_tokens': 74, 'output_tokens': 7, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
{'response': 'snippet: Je bent welkom. Heb je even geen geld om eten te kopen? '
             'Geen zorgen. Dat kan iedereen overkomen. Wanneer kom ik in '
             'aanmerking? Als je je aanmeldt, krijg je meteen boodsch

To test out whether each individual node works, you can call them separately:

In [85]:
query_understanding_node({"response": "I am hungry"})

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


{'response': QueryAnalysis(original_query='I am hungry', query_type='clear', domains=['Health & Wellbeing'], emotional_state='Neutral', language='English')}

In [86]:
web_agent_node({"response": QueryAnalysis(original_query='I am hungry', query_type='clear', domains=['Health & Wellbeing'], emotional_state='Neutral', language='English')})

This is the LLM query search response:
AIMessage(content='Find food assistance near me.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 74, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b2335398-6c21-4e7f-8be9-9e98ae89d265-0', usage_metadata={'input_tokens': 74, 'output_tokens': 7, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


{'response': 'snippet: Je bent welkom. Heb je even geen geld om eten te kopen? Geen zorgen. Dat kan iedereen overkomen. Wanneer kom ik in aanmerking? Als je je aanmeldt, krijg je meteen boodschappen mee. Dat doen we minimaal voor een maand. Ondertussen bespreken en bekijken we de hele papierwinkel binnen drie maanden na je aanmelding. We kijken […], title: Kom ik in aanmerking? - Vereniging van Voedselbanken Nederland, link: https://voedselbankennederland.nl/ik-zoek-hulp/kom-ik-in-aanmerking/, snippet: Jij kunt dan ook een enorme bijdrage leveren aan het helpen van meer mensen. Zoals in het eerste artikel van deze Vitamine te lezen, heb je daarmee niet alleen impact op een gezin maar op de hele samenleving. Bovendien is het een enorme stimulans voor onze 14.000 vrijwilligers. Het werk dat ze doen is letterlijk […], title: Voor € 25,- help je een gezin een hele maand - Vereniging van ..., link: https://voedselbankennederland.nl/voor-e25-per-maand-help-je-een-gezin-aan-een-voedselpakket/

Now let's turn this into a more interesting graph!

In [79]:
def emergency_node(state):
    """Returns emergency contact information"""
    whatsapp_number = "ADD RED CROSS WHATSAPP # HERE"
    return {
        "response": [
            {
                "role": "assistant",
                "content": f"""  
                    This seems urgent and like you need immediate assistance. Please contact the Red Cross directly at this number {whatsapp_number}  
                    to get help immediately.  
                    For any medical emergency please contact 112.  
                    """
            }
        ]
    }

In [80]:
def route_by_query_type(state):
    """
    Routes to appropriate node based on query analysis
    In query_understanding QueryAnalysis.query_type Literal["clear", "needs_clarification", "emergency"]
    """
    analysis = state["response"]
    # Route based on query type
    if analysis.query_type == "clear":
        return "web_agent"
    elif analysis.query_type == "emergency":
        return "emergency"
        
workflow = StateGraph(ConversationBox)
workflow.add_node("query_understanding", query_understanding_node)
workflow.add_node("emergency", emergency_node)
workflow.add_node("web_agent", web_agent_node)
workflow.add_edge(START, "query_understanding")
workflow.add_conditional_edges(
        "query_understanding",
        route_by_query_type,
        {
            "web_agent": "web_agent",
            "emergency": "emergency",
        }
    )
workflow.add_edge("emergency", END)
workflow.add_edge("web_agent", END)
graph = workflow.compile()

In [81]:
graph.invoke({"response": "I am hungry"})

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


This is the LLM query search response:
AIMessage(content='Healthy snack options for hunger mood upsurgence.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 74, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4f3dd918-7f9e-4308-b606-72422af8f3a8-0', usage_metadata={'input_tokens': 74, 'output_tokens': 10, 'total_tokens': 84, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


{'response': "snippet: Je bent welkom. Heb je even geen geld om eten te kopen? Geen zorgen. Dat kan iedereen overkomen. Wanneer kom ik in aanmerking? Als je je aanmeldt, krijg je meteen boodschappen mee. Dat doen we minimaal voor een maand. Ondertussen bespreken en bekijken we de hele papierwinkel binnen drie maanden na je aanmelding. We kijken […], title: Kom ik in aanmerking? - Vereniging van Voedselbanken Nederland, link: https://voedselbankennederland.nl/ik-zoek-hulp/kom-ik-in-aanmerking/, snippet: Jij kunt dan ook een enorme bijdrage leveren aan het helpen van meer mensen. Zoals in het eerste artikel van deze Vitamine te lezen, heb je daarmee niet alleen impact op een gezin maar op de hele samenleving. Bovendien is het een enorme stimulans voor onze 14.000 vrijwilligers. Het werk dat ze doen is letterlijk […], title: Voor € 25,- help je een gezin een hele maand - Vereniging van ..., link: https://voedselbankennederland.nl/voor-e25-per-maand-help-je-een-gezin-aan-een-voedselpakket/

In [82]:
response = graph.invoke({"response": "Please help me!"})
pprint(response['response'])

c:\Users\SabinaFirtala\anaconda3\envs\lizzy_nlp\Lib\site-packages\langchain_openai\chat_models\base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


[{'content': '  \n'
             '                    This seems urgent and like you need '
             'immediate assistance. Please contact the Red Cross directly at '
             'this number ADD RED CROSS WHATSAPP # HERE  \n'
             '                    to get help immediately.  \n'
             '                    For any medical emergency please contact '
             '112.  \n'
             '                    ',
  'role': 'assistant'}]


## <a id='toc1_5_'></a>[Conclusion](#toc0_)
- **LLM agents** enable intelligent text-based automation.
- **LangChain** enhances LLMs with tools, memory, and structured pipelines.
- **LangGraph** facilitates multi-agent workflows for complex AI interactions.

With these tools, you can build powerful AI-driven applications. 🚀

# <a id='toc2_'></a>[Extra](#toc0_)

## <a id='toc2_1_'></a>[Tool-Using Agent](#toc0_)
An agent that integrates with APIs or tools (e.g., calculator, weather API, web scraper).

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain_core.tools import tool

@tool # When we use the scraping tool, the output will be compatible with LangChain
def web_scrape_tool(url):
    """Sends simple request to websites and retrieves the answer"""
    try:
        response = requests.get(url)
        content = BeautifulSoup(response.text, 'html.parser')
        content_list = [p.text.replace('\n', '') for p in content.find_all('p')]
        final_content = "\n".join(content_list)
        return final_content[:1000]  # Limit to first 1000 characters
    except Exception as e:
        return f"Error scraping {url}: {str(e)}"

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

example_url = "https://en.wikipedia.org/wiki/LangChain"
message_history = [
    AIMessage("You are a web research assistant who can scrape and summarize web content."),
    HumanMessage(f"Can you summarize the info on this webpage: {example_url}?"),
]
llm = init_chat_model("gpt-3.5-turbo", model_provider="openai", temperature=0).bind_tools([web_scrape_tool])
response = llm.invoke(message_history)
response

In [ ]:
for tool_call in response.tool_calls:
    if tool_call['name'].lower() == 'web_scrape_tool':
        url = tool_call['args']['url']
        tool_output = web_scrape_tool(url)
        tool_message = ToolMessage(
            content=tool_output, 
            tool_call_id=tool_call['id']
        )
        message_history.append(response)
        message_history.append(tool_message)

message_history

In [ ]:
llm.invoke(message_history)

As you can see, we didn't get an answer to our query. Instead, we were able to extract the URL from the user's prompt. To also get the result, we need to pass the tool input as a message!

## <a id='toc2_2_'></a>[RAG Agent](#toc0_)

RAG Agents require a bit more in-depth view over vector databases and embeddings but you can find a tutorial [here](https://www.kdnuggets.com/implement-agentic-rag-using-langchain-part-2).